In [2]:
# Load libraries 

import numpy as np
import matplotlib.pyplot as plt 
import geopandas as gpd
import rioxarray as rioxr
from shapely.geometry import Polygon

from pystac_client import Client # To access STAC catalogs

import planetary_computer # To sign items from MPC STAC

from IPython.display import Image # For nicer image display

You will work with two datasets for this task:

Biodiversity Intactness Index (BII) Time Series Access the io-biodiversity collection from the Microsoft Planetary Computer STAC catalog. Use the 2017 and 2020 rasters covering the Phoenix subdivision. For the bounding box, use the following coordinates:
[-112.826843, 32.974108, -111.184387, 33.863574]

Phoenix Subdivision Shapefile Download the Phoenix subdivision polygon from the Census County Subdivision shapefiles for Arizona.

In [4]:
# Access MPC catalog 
catalog = Client.open(
"https://planetarycomputer.microsoft.com/api/stac/v1",
modifier=planetary_computer.sign_inplace
)

In [10]:
# Get Biodiversity collections
collections = list(catalog.get_collections()) # Make list of collection names 

In [13]:
# Access Biodiversity collection using its id: io-biodiversity
bio_collection = catalog.get_child('io-biodiversity')
bio_collection

<CollectionClient id=io-biodiversity>

In [35]:
# Create bounding box 
bbox = {
    "type":"Polygon",
    "coordinates":[
        [
         [-112.826843, 32.974108],
         [-112.826843, 33.863574],
         [-111.184387, 33.863574],
         [-111.184387, 32.974108],
         [-112.826843, 32.974108]
        ]
    ]
}

In [38]:
# Catalog search for 2017 
search = catalog.search(
    collections = ['io-biodiversity'],
    intersects = bbox,
    datetime = '2017-01-01'
)

search

In [39]:
items = search.item_collection()

In [40]:
items

In [41]:
# Get item in catalog search
item = items[0]

In [56]:
# Explore the data
print('ID', item.id)

print(item.properties)

print(item.assets)

ID bii_2017_34.74464974521749_-115.38597824385106_cog
{'datetime': None, 'proj:epsg': 4326, 'proj:shape': [7992, 7992], 'end_datetime': '2017-12-31T23:59:59Z', 'proj:transform': [0.0008983152841195215, 0.0, -115.38597824385106, 0.0, -0.0008983152841195215, 34.74464974521749, 0.0, 0.0, 1.0], 'start_datetime': '2017-01-01T00:00:00Z'}
{'data': <Asset href=https://pcdata01euw.blob.core.windows.net/impact/bii-v1/bii_2017/bii_2017_34.74464974521749_-115.38597824385106_cog.tif?st=2025-11-26T00%3A41%3A03Z&se=2025-11-27T01%3A26%3A03Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-11-26T20%3A50%3A21Z&ske=2025-12-03T20%3A50%3A21Z&sks=b&skv=2025-07-05&sig=U3IV1xVLkHo5W5XiGNqaW/dRSMQxttWE8N9elh03kbk%3D>, 'tilejson': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=io-biodiversity&item=bii_2017_34.74464974521749_-115.38597824385106_cog&assets=data&tile_format=png&colormap_name=io-bii

In [57]:
# Preview the data
Image(url=item.assets['rendered_preview'].href,width=500)

In [59]:
az_2017 = rioxr.open_rasterio(item.assets['data'].href)
az_2017 

<xarray.DataArray (band: 1, y: 7992, x: 7992)> Size: 255MB
[63872064 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 64kB -115.4 -115.4 -115.4 ... -108.2 -108.2 -108.2
  * y            (y) float64 64kB 34.74 34.74 34.74 34.74 ... 27.57 27.57 27.57
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [60]:

years = ['2017-01-01','2020-01-01']

az_dict= {}

for year in years:
    # Catalog search for 2017 
    search = catalog.search(
    collections = ['io-biodiversity'],
    intersects = bbox,
    datetime = year
)
    items = search.item_collection()

    for item in items:
        
        Image(url=item.assets['rendered_preview'].href,width=500)

        if year not in az_dict:
            az_dict[year].append(rioxr.open_rasterio(item.assets['data'].href))


KeyError: '2017-01-01'